# Instalar dependencias

In [ ]:
!pip install neo4j
!pip install networkx
!pip install win10toast
!pip install graphdatascience[networkx]
!pip install ipywidgets

# Sincrono vs Asincrono
Repetiremos la misma query de manera sincrona y asincrona

## Ejecutar una query no-concurrente

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

query = """
MATCH (e1:Estacion)-[:TRAMO {linea:$line}]->(:Estacion) WITH e1 as l8
OPTIONAL MATCH (l8)-[t:TRAMO]->(:Estacion)
WHERE t.linea <> $line 
RETURN l8.nombre as estacion, collect(distinct t.linea) as transbordo
"""

with GraphDatabase.driver('bolt://localhost:7687', auth=("neo4j", "BDII2023")) as driver:
        with driver.session() as session:
                results = session.run(query, line="8")
                display(pd.DataFrame(results, columns=results.keys()))


## Ejecutar una query concurrente con asyncio
Ejecutaremos una query en version no bloqueante, sinembargo, haremos que jupyter espere a que termine de realizar la query en la base de datos para obtener los resultados. Se incluye un delay para que el efecto se note mas. Notese que en este caso la celda tarda en ejecutarse 5 segundos y nos muestra los resultados de la recomendacion.

In [ ]:
import asyncio
from neo4j import AsyncGraphDatabase

async def do_recommendation(line):

    query = """
    MATCH (e1:Estacion)-[:TRAMO {linea:$line}]->(:Estacion) WITH e1 as l8
    OPTIONAL MATCH (l8)-[t:TRAMO]->(:Estacion)
    WHERE t.linea <> $line 
    RETURN l8.nombre as estacion, collect(distinct t.linea) as transbordo
    """
    
    async with AsyncGraphDatabase.driver('bolt://localhost:7687', auth=("neo4j", "BDII2023")) as driver:
        async with driver.session() as session:
            results = await session.run(query, line=line)
            data = await results.data()
            await asyncio.sleep(5)
            return pd.DataFrame(data, columns=data[0].keys())

task = asyncio.create_task(do_recommendation("8"))
df = await task
display(df)

## Ejecutar una query con call_back 
En vez de esperar a que se termine la tarea se asigna un callback que mandara una notificacion al usuario.
Se incluye un delay en la funcion que realiza la query a la base de datos para que se note el cambio. Notese que en este caso la celda tarda en ejecutarse 0 segundos y no muestra la recomendacion. Pasados 5 segundos se le mostrará al usuario una notificacion con los resultados.

In [ ]:
import pandas as pd
from win10toast import ToastNotifier
import asyncio
from neo4j import AsyncGraphDatabase

async def do_recommendation(line):

    query = """
    MATCH (e1:Estacion)-[:TRAMO {linea:$line}]->(:Estacion) WITH e1 as l8
    OPTIONAL MATCH (l8)-[t:TRAMO]->(:Estacion)
    WHERE t.linea <> $line 
    RETURN l8.nombre as estacion, collect(distinct t.linea) as transbordo
    """
    
    async with AsyncGraphDatabase.driver('bolt://localhost:7687', auth=("neo4j", "BDII2023")) as driver:
        async with driver.session() as session:
            results = await session.run(query, line=line)
            data = await results.data()
            await asyncio.sleep(5)
            return pd.DataFrame(data, columns=data[0].keys())
        
def notify(df):
    toaster = ToastNotifier()
    
    # string with dataframe data
    df_string = df.to_string()
    toaster.show_toast("Recommendation", df_string, duration=20)

task = asyncio.create_task(do_recommendation("8"))
task.add_done_callback(lambda future: notify(future.result()))


# Traer grafo a python con networkx
En este caso no queremos traer informacion tabular, sino informacion en forma de grafo. Para ellos transformaremos una query que devuelve nodos y aristas en un Objeto de tipo Graph de la libreria NetworkX

In [ ]:
import networkx as nx
from neo4j import GraphDatabase

def run_query(query, **kwargs):
    with GraphDatabase.driver('bolt://localhost:7687', auth=("neo4j", "BDII2023")) as driver:
        with driver.session() as session:
                result = list(session.run(query, **kwargs))
                return result

query_nodes = """
MATCH (e:Estacion) WHERE (e)-[:TRAMO {linea:$line}]->()
RETURN e.codigo as id, {name: e.nombre, x: e.x, y: e.y} as p
"""

query_edges = """
MATCH (e1:Estacion)-[t:TRAMO {linea:$line}]->(e2:Estacion) 
RETURN e1.codigo as source, e2.codigo as target, {length: t.longitud} as w
"""

g = nx.Graph()
g.add_nodes_from([(r['id'], r['p']) for r in run_query(query_nodes, line="8")])
g.add_edges_from([(r['source'], r['target'], r['w']) for r in run_query(query_edges, line="8")])

print(f"Number of nodes: {g.number_of_nodes()}")
print(f"Node attributes: {list(list(g.nodes(data=True))[0][1].keys())}")
print(f"Number of edges: {g.number_of_edges()}")
print(f"Edge attributes: {list(list(g.edges(data=True))[0][2].keys())}")
print(f"Number of connected components: {nx.number_connected_components(g)}")